# Synchronous Generator dq 7th order

## Run Simulations

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

## DP ODE model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odODE_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQODE('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## EMT ODE Model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'EMT_SynGenDq7odODE_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.emt.ph3.SynchronGeneratorDQODE('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.emt.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.emt.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.emt.SimNode.gnd, n1])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.do_init_from_nodes_and_terminals(False)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## DP Trapez model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odTrapez_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQTrapez('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_gen', 'i_intf', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## EMT Trapez model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'EMT_SynGenDq7odTrapez_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.emt.ph3.SynchronGeneratorDQTrapez('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    init_mech_power)

res = dpsimpy.emt.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.emt.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.emt.SimNode.gnd, n1])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## Three Phase Fault - DPsim ODE model

In [ ]:
work_dir = 'logs/DP_SynGenDq7odODE_ThreePhFault/'
log_name = 'DP_SynGenDq7odODE_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_ode_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
ts_dpsim_ode_tpf_emt = ts.frequency_shift_list(ts_dpsim_ode_tpf, 60)

In [ ]:
ts_dpsim_ode_tpf_phasors = ts.phasors(ts_dpsim_ode_tpf)
for node, phasor in ts_dpsim_ode_tpf_phasors.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

In [ ]:
work_dir = 'logs/EMT_SynGenDq7odODE_ThreePhFault/'
log_name = 'EMT_SynGenDq7odODE_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_emt_ode_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
plt.plot(ts_dpsim_ode_tpf_emt['v1_0_shift'].time, ts_dpsim_ode_tpf_emt['v1_0_shift'].values)
plt.plot(ts_dpsim_ode_tpf_emt['v1_1_shift'].time, ts_dpsim_ode_tpf_emt['v1_1_shift'].values)
plt.plot(ts_dpsim_ode_tpf_emt['v1_2_shift'].time, ts_dpsim_ode_tpf_emt['v1_2_shift'].values)
plt.show()

In [ ]:
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_0_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_0_shift'].values)
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_1_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_1_shift'].values)
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_2_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_2_shift'].values)
plt.plot(ts_dpsim_emt_ode_tpf['i_gen_0'].time, ts_dpsim_emt_ode_tpf['i_gen_0'].values)
plt.plot(ts_dpsim_emt_ode_tpf['i_gen_1'].time, ts_dpsim_emt_ode_tpf['i_gen_1'].values)
plt.plot(ts_dpsim_emt_ode_tpf['i_gen_2'].time, ts_dpsim_emt_ode_tpf['i_gen_2'].values)
plt.show()

## Three Phase Fault - DPsim Trapez

In [ ]:
work_dir = 'logs/DP_SynGenDq7odTrapez_ThreePhFault/'
log_name = 'DP_SynGenDq7odTrapez_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_trapez_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')
ts_dpsim_trapez_tpf_emt = ts.frequency_shift_list(ts_dpsim_trapez_tpf, 60)

In [ ]:
work_dir = 'logs/EMT_SynGenDq7odTrapez_ThreePhFault/'
log_name = 'EMT_SynGenDq7odTrapez_ThreePhFault'
print(work_dir + log_name + '.csv')
ts_dpsim_emt_trapez_tpf = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].values)
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_1_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_1_shift'].values)
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_2_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_2_shift'].values)
plt.plot(ts_dpsim_emt_trapez_tpf['i_gen_0'].time, ts_dpsim_emt_trapez_tpf['i_gen_0'].values)
plt.plot(ts_dpsim_emt_trapez_tpf['i_gen_1'].time, ts_dpsim_emt_trapez_tpf['i_gen_1'].values)
plt.plot(ts_dpsim_emt_trapez_tpf['i_gen_2'].time, ts_dpsim_emt_trapez_tpf['i_gen_2'].values)
plt.show()

## Three Phase Fault - Simulink

In [ ]:
# download Simulink log file from RWTH gitlab
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Simulink/SynchronousGenerator/SynGen_dq_7th_ThreePhFault/SL_SynGenDq7od_ThreePhFault_v_i.csv'
local_file = 'reference-results/SL_SynGenDq7od_ThreePhFault_v_i.csv'
urllib.request.urlretrieve(url, local_file) 

import villas.dataprocessing.readtools as rt
import matplotlib.pyplot as plt 
ts_sl_tpf = rt.read_timeseries_simulink(local_file)

In [ ]:
plt.plot(ts_sl_tpf['v1'].time, ts_sl_tpf['v1'].values)
plt.plot(ts_sl_tpf['v2'].time, ts_sl_tpf['v2'].values)
plt.plot(ts_sl_tpf['v3'].time, ts_sl_tpf['v3'].values)
plt.xlim(0,0.3)

plt.figure()
plt.plot(ts_sl_tpf['i1'].time, ts_sl_tpf['i1'].values)
plt.plot(ts_sl_tpf['i2'].time, ts_sl_tpf['i2'].values)
plt.plot(ts_sl_tpf['i3'].time, ts_sl_tpf['i3'].values)
plt.xlim(0,0.3)

## DPsim ODE and Simulink

In [ ]:
#%matplotlib qt
import matplotlib.pyplot as plt
ts_sl_tpf_i1 = ts('i1', ts_sl_tpf['i1'].time, -ts_sl_tpf['i1'].values)
ts_sl_tpf_i2 = ts('i2', ts_sl_tpf['i2'].time, -ts_sl_tpf['i2'].values)
ts_sl_tpf_i3 = ts('i3', ts_sl_tpf['i3'].time, -ts_sl_tpf['i3'].values)

#plt.figure(figsize=(20,10))
plt.plot(ts_sl_tpf_i1.time, ts_sl_tpf_i1.values, color = '#939393ff', label = 'ia_sl')
plt.plot(ts_sl_tpf_i2.time, ts_sl_tpf_i2.values, color = '#939393ff', label = 'ib_sl')
plt.plot(ts_sl_tpf_i3.time, ts_sl_tpf_i3.values, color = '#939393ff', label = 'ic_sl')
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_0_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_0_shift'].values, linestyle='-.', color = '#d62728ff', label = 'ia_dpsim')
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_1_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_1_shift'].values, linestyle='-.', color = '#0055ffff', label = 'ib_dpsim')
plt.plot(ts_dpsim_ode_tpf_emt['i_gen_2_shift'].time, ts_dpsim_ode_tpf_emt['i_gen_2_shift'].values, linestyle='-.', color = '#00aa7fff', label = 'ic_dpsim')
plt.xlabel('time (s)')
plt.ylabel('voltage (V)')
plt.xlim(0.05,0.25)
#plt.ylim(-1.6e5,1.6e5)
#plt.savefig('syngen_fault_ode.pdf')

In [ ]:
plt.plot(ts_dpsim_ode_tpf_phasors['i_gen_0']['phase'].time, ts_dpsim_ode_tpf_phasors['i_gen_0']['phase'].values)
plt.xlim([0.09,0.21])

plt.figure()
plt.plot(ts_dpsim_ode_tpf['wr_gen'].time, ts_dpsim_ode_tpf['wr_gen'].values * 60)
plt.xlim([0.09,0.21])

In [ ]:
import numpy as np
diff = ts_sl_tpf_i1.values - ts_dpsim_ode_tpf_emt['i_gen_0_shift'].values[:-1]
print(np.max(diff[:4000]))
assert np.max(diff[:4000]) < 28.5

## DPsim Trapez and Simulink

In [ ]:
ts_sl_tpf_i1 = ts('i1', ts_sl_tpf['i1'].time, -ts_sl_tpf['i1'].values)
ts_sl_tpf_i2 = ts('i2', ts_sl_tpf['i2'].time, -ts_sl_tpf['i2'].values)
ts_sl_tpf_i3 = ts('i3', ts_sl_tpf['i3'].time, -ts_sl_tpf['i3'].values)

#plt.figure(figsize=(20,10))
plt.plot(ts_sl_tpf_i1.time, ts_sl_tpf_i1.values)
plt.plot(ts_sl_tpf_i2.time, ts_sl_tpf_i2.values)
plt.plot(ts_sl_tpf_i3.time, ts_sl_tpf_i3.values)
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].values)
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_1_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_1_shift'].values)
plt.plot(ts_dpsim_trapez_tpf_emt['i_gen_2_shift'].time, ts_dpsim_trapez_tpf_emt['i_gen_2_shift'].values)
plt.show()

In [ ]:
import numpy as np
diff = ts_sl_tpf_i1.values - ts_dpsim_trapez_tpf_emt['i_gen_0_shift'].values[:-1]
print(np.max(diff[:4000]))
assert np.max(diff[:4000]) < 30.6